## Import & Install Statements

In [ ]:
# Import statements

import pandas as pd
import numpy as np
import re
import random

import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords

nltk.download('vader_lexicon')
nltk.download('stopwords')

sia = SentimentIntensityAnalyzer()
stop_words = set(stopwords.words('english'))

import spacy
nlp = spacy.load('en_core_web_sm')

from sklearn.model_selection import train_test_split

from transformers import T5Tokenizer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Important Stop Words List Creation & Feedback Template Generation

In [ ]:
# Create a list of important contextual stopwords that we don't want to get rid of.

important_stopwords = {'the', 'you', 'i', 'to', 'for', 'and', 'your', 'that', 'a', 'with', 'is', 'can', 'have', 'it', 'please', 'me', 'my', 'on', 'of', 'we', 'was', 'there', 'in', 'if', 'will', 'this', 're', 'our', 'any', 'any', 'or'}
stop_words = stop_words - important_stopwords

In [ ]:
# Create a series of evaluation templates for the model to apply to conversations.

feedback_templates = {
    "empathy_shown": [
        "The agent ackowledged the customer's emotions well.",
        "The agent displayed strong empathy toward the customer's situation.",
        "Good job addressing the customer's feelings with empathy.",
        "The agent showed genuine concern for the customer's emotions.",
        "Empathy was clearly demonstrated by the agent.",
        "The agent was considerate of the customer's stress.",
        "The response demonstrated the agent's understanding of the customer's frustration.",
        "The agent showed sympathy for the customer's predicament.",
        "The agent's reponse was empathetic and acknowledged the customer's difficulty.",
        "The agent showed understanding and compassion for the customer's issue."
    ],
    "clarity_instructions": [
        "The instructions provided were clear and concise.",
        "The agent provided clear and actionable steps.",
        "The guidance was straighforward and easy to follow.",
        "The customer received detailed and comprehensible instructions.",
        "The agent's directions were simple and easy to understand.",
        "The agent provided well-structured and easy-to-follow steps.",
        "The instructions given were precise and to the point.",
        "The agent provided easy-to-digest steps for the customer.",
        "The guidance was organized logically, making it easy for the customer to proceed.",
        "The agent offered actionable and digestable guidance for resolution."
    ],
    "tone_matching": [
        "The agent's tone was well-aligned with the customer's sentiment.",
        "The response tone was appropriately balanced with the customer's emotions.",
        "The agent's tone resonated well with the customer's mood.",
        "The agent maintained a supportive and understanding tone.",
        "The agent's tone was suitable for the customer's level of frustration.",
        "The tone conveyed a sense of urgency that matched the customer's needs.",
        "The agent responded in a measured way that reflected the customer's initial message.",
        "The tone conveyed a helpful attitude, showing attentiveness to the customer's concerns."
    ],
    "missed_concern":[
        "The agent may have overlooked a primary concern of the customer.",
        "There is a chance the agent missed addressing a key part of the customer's query.",
        "The agent didn't fully cover the main point of the customer's concern.",
        "The response could have been more thorough in addressing all the customer's issues.",
        "The agent could have paid more attention to an important aspect of the customer's question.",
        "The agent did not address an implied question from the customer.",
        "The customer's underlying issue wasn't fully explored in the response.",
        "The response might have benefitted from more proactive anticipation of follow-up concerns.",
        "The agent didn't ask clarifying questions that could have addressed the issue comprehensively."
    ]
}

## Adding Feedback Templates to the data

In [ ]:
customer_df = pd.read_csv('/content/customer_service_df_with_daily_dialog_labels.csv')

In [ ]:
# Take the feedback templates above and apply them to the data based on context.

def generate_contextual_feedback(row):
  feedback = []
  if row['issue_complexity'] == 'high':
    feedback.append("The customer is dealing with a complex issue. Detailed and empathatic guidance is necessary.")
  elif row['issue_complexity'] == 'medium':
    feedback.append("The customer issue is moderately complex. Clarity and step-by-step assistance are beneficial.")
  else:
    feedback.append("This is a straightforward issue. Quick, clear guidance is effective.")

  customer_sentiment = sia.polarity_scores(str(row['input']))['compound']
  agent_sentiment = sia.polarity_scores(str(row['response']))['compound']

  if customer_sentiment < -0.3 and agent_sentiment > 0.3:
    feedback.append("The agent's response tone was positive despite customer frustration, which may not be appropriate.")
  elif customer_sentiment < -0.3 and agent_sentiment < 0:
    feedback.append("The agent's response acknowledged the customer's frustration appropriately.")
  else:
    feedback.append("The agent's response tone matched the customer's sentiment well.")
  return " ".join(feedback)

In [ ]:
# In order to not have the data repeat the same feedback for every row, use the random function to create a more variety for the model to better learn from.

def generate_diverse_feedback(row):
  feedback = []

  feedback.append(generate_contextual_feedback(row))

  feedback.append(random.choice(feedback_templates["empathy_shown"]))
  feedback.append(random.choice(feedback_templates["clarity_instructions"]))
  feedback.append(random.choice(feedback_templates["tone_matching"]))
  if random.random() < 0.5:
    feedback.append(random.choice(feedback_templates["missed_concern"]))
  return " ".join(feedback)

In [ ]:
# Apply the feedback templates to the data.

def prepare_finetuning_data(row):
  input_text = f"Conversation: {row['input']} ||| {row['response']}. Issue: {row['issue_category']}. Complexity: {row['issue_complexity']}."

  diverse_feedback = generate_diverse_feedback(row)
  return pd.Series([input_text, diverse_feedback], index = ['input_for_model', 'diverse_output_for_model'])

finetuning_data = customer_df.apply(prepare_finetuning_data, axis = 1)

In [ ]:
# Add issue complexity & customer sentiment to finetuning data as that's now the dataframe we're going to be working from.
finetuning_data['issue_complexity'] = customer_df['issue_complexity']
finetuning_data['customer_sentiment'] = customer_df['customer_sentiment']

In [ ]:
# # Save finetuning data to a csv file.
# finetuning_data.to_csv('enhanced_finetuning_data_with_diversified_feedback.csv')

In [ ]:
# Reload the dataframe; keep the name as customer_df_diversified_feedback for continuity with previous code.
customer_df_diversified_feedback = pd.read_csv('/content/enhanced_finetuning_data_with_diversified_feedback.csv')

## Feature Engineering

In [ ]:
# Create a text cleaning function.
def clean_text(text):
  if not isinstance(text, str):
    return ""
  text = text.lower()
  text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags = re.MULTILINE)
  text = re.sub(r'[^a-z\s]', '', text)
  text = re.sub(r'\s+', ' ', text).strip()
  text = ' '.join([word for word in text.split() if word not in stop_words])
  return text

In [ ]:
# Clean the text for input and output and create two new columns instead of overriding the original data; while we could've just applied it to the columns instead of creating new ones, this gives us flexibility for data prep for training the model and we can easily drop the original columns as needed.
customer_df_diversified_feedback['cleaned_input'] = customer_df_diversified_feedback['input_for_model'].apply(clean_text)
customer_df_diversified_feedback['cleaned_output'] = customer_df_diversified_feedback['diverse_output_for_model'].apply(clean_text)

In [ ]:
# Create two new features that give us the length of the new input & output columns.

customer_df_diversified_feedback['adjusted_input_length'] = customer_df_diversified_feedback['cleaned_input'].apply(lambda x: len(x.split()))
customer_df_diversified_feedback['adjusted_output_length'] = customer_df_diversified_feedback['cleaned_output'].apply(lambda x: len(x.split()))

In [ ]:
# Create a function that checks how the input & output match in terms of keyword matches.
def keyword_match_feedback(row):
  input_keywords = set(row['cleaned_input'].split())
  output_keywords = set(row['cleaned_output'].split())
  match_count = len(input_keywords.intersection(output_keywords))
  return match_count/len(input_keywords) if len(input_keywords) > 0 else 0

customer_df_diversified_feedback['keyword_match_ratio'] = customer_df_diversified_feedback.apply(keyword_match_feedback, axis = 1)

In [ ]:
# Scale the keyword match ratios to a 1-10 scale and a 1-100 scale.
def scale_keyword_match_ratio_1_10(ratio):
  return((ratio + 1)/2 * 9 + 1)
def scale_keyword_match_ratio_1_100(ratio):
  return((ratio + 1)/2 * 99 + 1)

customer_df_diversified_feedback['keyword_match_ratio_1_10'] = customer_df_diversified_feedback['keyword_match_ratio'].apply(scale_keyword_match_ratio_1_10)
customer_df_diversified_feedback['keyword_match_ratio_1_100'] = customer_df_diversified_feedback['keyword_match_ratio'].apply(scale_keyword_match_ratio_1_100)

In [ ]:
# Add sentiment scores to the input.
customer_df_diversified_feedback['input_sentiment'] = customer_df_diversified_feedback['input_for_model'].apply(lambda x: sia.polarity_scores(x)['compound'])

In [ ]:
# Scale the sentiment scores to a 1-10 and a 1-100 scale.

def scale_sentiment_1_10(score):
  return((score + 1)/2 * 9 + 1)
def scale_sentiment_1_100(score):
  return((score + 1)/2 * 99 + 1)

customer_df_diversified_feedback['input_sentiment_1_10'] = customer_df_diversified_feedback['input_sentiment'].apply(scale_sentiment_1_10)
customer_df_diversified_feedback['input_sentiment_1_100'] = customer_df_diversified_feedback['input_sentiment'].apply(scale_sentiment_1_100)

In [ ]:
# Add markers for indicating politeness & empathy.

def add_politeness_empathy_markers(text):
  politeness_markers = ['please', 'thank', 'thanks', 'sorry', 'apologies', 'kindly', 'sure', 'may']
  empathy_markers = ['understand', 'empathize', 'frustration', 'concern', 'appreciate', 'unfortunate', 'help', 'assist', 'concern']

  politeness_count = sum(text.count(marker) for marker in politeness_markers)
  empathy_count = sum(text.count(marker) for marker in empathy_markers)

  return politeness_count, empathy_count

customer_df_diversified_feedback[['politeness_count_input', 'empathy_count_input']] = customer_df_diversified_feedback['cleaned_input'].apply(
    lambda x: pd.Series(add_politeness_empathy_markers(x))
)

customer_df_diversified_feedback[['politeness_count_output', 'empathy_count_output']] = customer_df_diversified_feedback['cleaned_output'].apply(
    lambda x: pd.Series(add_politeness_empathy_markers(x))
)

In [ ]:
# Take out entities for input & output for applying it in the function below.

def extract_entities(text):
  doc = nlp(text)
  entities = [ent.text for ent in doc.ents]
  return entities

customer_df_diversified_feedback['input_entities'] = customer_df_diversified_feedback['cleaned_input'].apply(extract_entities)
customer_df_diversified_feedback['output_entities'] = customer_df_diversified_feedback['cleaned_output'].apply(extract_entities)

In [ ]:
# Taking out the entities allows us to create an entity match ratio. The entity match ratio essentially tells us how well agents address things that the customer mentions in the conversation.

def entity_match_ratio(row):
  input_entities = set(row['input_entities'])
  output_entities = set(row['output_entities'])

  if len(input_entities) == 0:
    return 0
  return len(input_entities.intersection(output_entities))/len(input_entities)

customer_df_diversified_feedback['entity_match_ratio'] = customer_df_diversified_feedback.apply(entity_match_ratio, axis = 1)

In [ ]:
# Save the feature-engineered data frame to a csv file, and then reload it below.
# customer_df_diversified_feedback.to_csv('customer_finetuning_data_preprocessed.csv', index = False)

## Creating the Fully Feature-Engineered Dataset

In [ ]:
finetuning_data = pd.read_csv('/content/customer_finetuning_data_preprocessed.csv')

In [ ]:
# Let's take a look at the full expanded data frame.
finetuning_data.columns

Index(['issue_area', 'issue_category', 'issue_sub_category',
       'issue_category_sub_category', 'customer_sentiment', 'product_category',
       'product_sub_category', 'issue_complexity', 'agent_experience_level',
       'agent_experience_level_desc', 'conversation_id',
       'full_conversation_iteratively_updated', 'dialogue_act_label',
       'emotion_label', 'response_sentiment', 'sentiment_label', 'input',
       'response', 'context', 'contextual_feedback', 'key_phrases_input',
       'key_phrases_response', 'keyword_feedback', 'dynamic_feedback',
       'input_for_model', 'output_for_model', 'diverse_output_for_model',
       'cleaned_input', 'cleaned_output', 'adjusted_input_length',
       'adjusted_output_length', 'keyword_match_ratio',
       'keyword_match_ratio_1_10', 'keyword_match_ratio_1_100',
       'input_sentiment', 'input_sentiment_1_10', 'input_sentiment_1_100',
       'cleaned_input_sentiment', 'cleaned_input_sentiment_1_10',
       'cleaned_input_sentiment_1_

In [ ]:
# Let's create a list of some of the columns that we think are important for this section.
columns_to_keep = ['input', 'response', 'issue_complexity', 'customer_sentiment', 'agent_experience_level', 'cleaned_output']

In [ ]:
'''
Preprocess the data for training. The goal here is to reduce the load on the model so that we're not giving it a highly complex dataset with a lot of columns that it has to juggle.
So let's add the metadata for the columns to keep list above to a column called input_text that's going to contain the conversation along with this metadata.
'''
def preprocess_for_training(row):
  input_text = (
      f"Customer Issue:\n{row['input']}\n\n"
      f"Agent Reponse:\n{row['response']}\n\n"
      f"Issue Complexity:\n{row['issue_complexity']}\n\n"
      f"Customer Sentiment:\n{row['customer_sentiment']}\n\n"
      f"Agent Experience Level:\n{row['agent_experience_level']}\n\n"
      f"Task: Evaluate the agent's performance in terms of empathy, clarity, and tone."
  )

  output_text = row['cleaned_output']
  return input_text, output_text

In [ ]:
# Apply the function to the entire dataset and turn each row into new input & output pairs for model training.
finetuning_data.loc[:, ['input_text', 'output_text']] = finetuning_data.apply(
    lambda row: preprocess_for_training(row), axis = 1, result_type = 'expand'
)

In [ ]:
# Read the dataset again for concatenation & merging.
finetuning_data_preprocessed = pd.read_csv('/content/customer_finetuning_data_preprocessed.csv')

In [ ]:
# Concatenate & merge the new input/output pairs to the original dataset
merged_finetuning_data = pd.concat([finetuning_data_preprocessed, finetuning_data[['input_text', 'output_text']]], axis = 1)

In [ ]:
# Apply the diverse feedback from above to the new output pair column that we just created.
merged_finetuning_data['output_text'] = finetuning_data.apply(generate_diverse_feedback, axis = 1)

In [ ]:
# Take one final look at the dataset columns.
print('Final Dataset Columns:')
merged_finetuning_data.columns

Final Dataset Columns:


Index(['issue_area', 'issue_category', 'issue_sub_category',
       'issue_category_sub_category', 'customer_sentiment', 'product_category',
       'product_sub_category', 'issue_complexity', 'agent_experience_level',
       'agent_experience_level_desc', 'conversation_id',
       'full_conversation_iteratively_updated', 'dialogue_act_label',
       'emotion_label', 'response_sentiment', 'sentiment_label', 'input',
       'response', 'context', 'contextual_feedback', 'key_phrases_input',
       'key_phrases_response', 'keyword_feedback', 'dynamic_feedback',
       'input_for_model', 'output_for_model', 'diverse_output_for_model',
       'cleaned_input', 'cleaned_output', 'adjusted_input_length',
       'adjusted_output_length', 'keyword_match_ratio',
       'keyword_match_ratio_1_10', 'keyword_match_ratio_1_100',
       'input_sentiment', 'input_sentiment_1_10', 'input_sentiment_1_100',
       'cleaned_input_sentiment', 'cleaned_input_sentiment_1_10',
       'cleaned_input_sentiment_1_

In [ ]:
# Take a look at the first 5 rows of the data frame.
merged_finetuning_data.head()

,issue_area,issue_category,issue_sub_category,issue_category_sub_category,customer_sentiment,product_category,product_sub_category,issue_complexity,agent_experience_level,agent_experience_level_desc,...,cleaned_input_sentiment_1_100,politeness_count_input,empathy_count_input,politeness_count_output,empathy_count_output,input_entities,output_entities,entity_match_ratio,input_text,output_text
0,Login and Account,Mobile Number and Email Verification,Verification requirement for mobile number or ...,Mobile Number and Email Verification -> Verifi...,neutral,Appliances,Oven Toaster Grills (OTG),medium,junior,"handles customer inquiries independently, poss...",...,99.54955,13,4,1,3,['tom'],[],0.0,NaN,The customer issue is moderately complex. Clar...
1,Cancellations and returns,Pickup and Shipping,Reasons for being asked to ship the item,Pickup and Shipping -> Reasons for being asked...,neutral,Electronics,Computer Monitor,less,junior,"handles customer inquiries independently, poss...",...,99.57925,9,5,0,0,"['alex', 'last week', 'a seconds', 'the next h...",[],0.0,NaN,"This is a straightforward issue. Quick, clear ..."
2,Cancellations and returns,Replacement and Return Process,Inability to click the 'Cancel' button,Replacement and Return Process -> Inability to...,neutral,Appliances,Juicer/Mixer/Grinder,medium,experienced,"confidently handles complex customer issues, e...",...,99.67825,13,5,0,1,"['sarah', 'first']",[],0.0,NaN,The customer issue is moderately complex. Clar...
3,Login and Account,Login Issues and Error Messages,Error message regarding exceeded attempts to e...,Login Issues and Error Messages -> Error messa...,neutral,Appliances,Water Purifier,less,inexperienced,"may struggle with ambiguous queries, rely on c...",...,99.70300,14,7,0,0,"['a minutes a minutes', 'a minute']",[],0.0,NaN,"This is a straightforward issue. Quick, clear ..."
4,Order,Order Delivery Issues,Delivery not attempted again,Order Delivery Issues -> Delivery not attempte...,negative,Electronics,Bp Monitor,medium,experienced,"confidently handles complex customer issues, e...",...,50.58910,7,3,0,3,"['sarah', 'david', 'today']",[],0.0,NaN,The customer issue is moderately complex. Clar...


In [ ]:
# Save the data frame to a csv file so that we can load it in the next codebase when we train the model!
# merged_finetuning_data.to_csv('finetuning_data_diversified_output_text.csv', index = False)

In [ ]:
match_ratios = customer_df.groupby('issue_complexity')[['keyword_match_ratio_1_10']].mean()
capitalized_rows_match_ratios = [row.capitalize() for row in match_ratios.index]
capitalized_columns_match_ratios = ['Issue Complexity', 'Keyword Match Ratio (1-10)']

# Combine row labels and data into the table data
table_data = [[row] + [f"{value:.2f}"] for row, value in zip(capitalized_rows_match_ratios, match_ratios.values.flatten())]

fig, ax = plt.subplots(figsize=(7.5, 1.75), dpi=350)
fig.suptitle('Table: Keyword Match Ratio vs. Issue Complexity', fontsize=16, y=1.05, ha='center')
ax.axis('off')
ax.axis('tight')

# Adjust column widths
col_widths = [0.3, 0.7]  # Adjust to desired proportions

table = ax.table(cellText=table_data,
                 colLabels=capitalized_columns_match_ratios,
                 cellLoc='center',
                 rowLoc='center',
                 colWidths=col_widths,  # Set column widths
                 bbox=[0, 0, 1, 1])
table.auto_set_font_size(False)
table.set_fontsize(14)

plt.tight_layout()
plt.subplots_adjust(top=0.85)
plt.show()
